Generate map data to be used on a custom Google Map https://www.google.com/mymaps


In [ ]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
print("Imported libs")

Imported libs


In [2]:
colocs = pl.read_csv("seen_colocs.csv")
colocs

colocationCenter,city,country
str,str,str
"""CFC""","""Chapeco""","""Brazil"""
"""SOF""","""Sofia""","""Bulgaria"""
"""MSP""","""Minneapolis""","""United States"""
"""IST""","""Istanbul""","""Turkey"""
"""MNL""","""Manila""","""Philippines"""
…,…,…
"""BEL""","""Belem""","""Brazil"""
"""PAT""","""Patna""","""India"""
"""MBA""","""Mombasa""","""Kenya"""


In [3]:
world_cities = pl.read_csv("worldcities.csv", ignore_errors=True)
world_cities

city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
str,str,f64,f64,str,str,str,str,str,i64,i64
"""Tokyo""","""Tokyo""",35.687,139.7495,"""Japan""","""JP""","""JPN""","""Tōkyō""","""primary""",37785000,1392685764
"""Jakarta""","""Jakarta""",-6.175,106.8275,"""Indonesia""","""ID""","""IDN""","""Jakarta""","""primary""",33756000,1360771077
"""Delhi""","""Delhi""",28.61,77.23,"""India""","""IN""","""IND""","""Delhi""","""admin""",32226000,1356872604
"""Guangzhou""","""Guangzhou""",23.13,113.26,"""China""","""CN""","""CHN""","""Guangdong""","""admin""",26940000,1156237133
"""Mumbai""","""Mumbai""",19.0761,72.8775,"""India""","""IN""","""IND""","""Mahārāshtra""","""admin""",24973000,1356226629
…,…,…,…,…,…,…,…,…,…,…
"""Al Jabīn""","""Al Jabin""",14.704,43.599,"""Yemen""","""YE""","""YEM""","""Raymah""","""admin""",null,1887910100
"""Nelspruit""","""Nelspruit""",-25.4745,30.9703,"""South Africa""","""ZA""","""ZAF""","""Mpumalanga""","""admin""",null,1710114438
"""Gqeberha""","""Gqeberha""",-33.9681,25.5981,"""South Africa""","""ZA""","""ZAF""","""Eastern Cape""","""""",null,1710000082


In [4]:
data = pl.read_csv("all_combined.csv", null_values=["null"])
data = data.with_columns(
    pl.from_epoch("timestamp", time_unit="s")
)
data

timestamp,balancerId,balancerIp,clientIpAccordingCloudflare,clientCountryAccordingCloudflare,balancerColocationCenter,targetDomain,scheme,httpVersion,tlsVersion,clientCountry,clientCity,clientAsn,clientNetwork,latencyTotal,latencyDNS,latencyTCP,latencyTLS,latencyFirstByte,latencyDownload
datetime[μs],str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,i64,i64,i64,i64,i64
2026-01-21 22:50:36,"""107f211""","""104.16.133.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",84,14,3,46,15,6
2026-01-21 22:50:36,"""107f156""","""104.16.133.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",78,11,1,18,37,11
2026-01-21 22:50:36,"""107f211""","""104.16.132.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",23,2,1,13,5,2
2026-01-21 22:50:37,"""107f179""","""104.16.132.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",27,1,1,14,10,1
2026-01-21 22:50:37,"""107f174""","""104.16.132.229""","""185.32.190.187""","""PT""","""LIS""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""PT""","""Lisbon""",62416,"""SAMPLING LINE-SERVICOS E INTER…",29,2,0,18,7,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2026-01-08 14:25:55,"""751f10""",null,"""113.11.240.232""","""VU""","""SUV""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""VU""","""Port-Vila""",9249,"""Telecom Vanuatu""",91,4,16,47,22,2
2026-01-08 14:25:56,"""751f5""",null,"""113.11.240.232""","""VU""","""SUV""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""VU""","""Port-Vila""",9249,"""Telecom Vanuatu""",93,4,16,49,22,2
2026-01-08 14:25:57,"""751f8""",null,"""113.11.240.232""","""VU""","""SUV""","""cloudflare.com""","""https""","""http/2""","""TLSv1.3""","""VU""","""Port-Vila""",9249,"""Telecom Vanuatu""",102,3,20,51,26,2


In [10]:
colocs_with_coords = colocs.join(world_cities, left_on="city", right_on="city_ascii", how="left")
colocs_with_coords = colocs_with_coords.select(["colocationCenter", "city", "country","lat", "lng", "iso3"]).unique(subset=["colocationCenter"])
colocs_with_coords

colocationCenter,city,country,lat,lng,iso3
str,str,str,f64,f64,str
"""HEL""","""Helsinki""","""Finland""",60.1708,24.9375,"""FIN"""
"""MAN""","""Manchester""","""United Kingdom""",53.479,-2.2452,"""GBR"""
"""OTP""","""Bucharest""","""Romania""",44.4325,26.1039,"""ROU"""
"""BEL""","""Belem""","""Brazil""",-1.4558,-48.5039,"""BRA"""
"""QRO""","""Queretaro""","""Mexico""",20.5875,-100.3928,"""MEX"""
…,…,…,…,…,…
"""BEY""","""Beirut""","""Lebanon""",33.8981,35.5058,"""LBN"""
"""TLL""","""Tallinn""","""Estonia""",59.4372,24.7453,"""EST"""
"""TPE""","""Taipei""","""Taiwan""",25.0375,121.5625,"""TWN"""


In [11]:
colocs_with_coords.filter(pl.col("lat").is_null())

colocationCenter,city,country,lat,lng,iso3
str,str,str,f64,f64,str


Should be empty, otherwise city can't be mapped to location

In [12]:
missing_colocs = (
    data.select(pl.col("balancerColocationCenter").unique())
    .join(colocs_with_coords, left_on="balancerColocationCenter", right_on="colocationCenter", how="full")
    .filter(pl.col("colocationCenter").is_null())
)
missing_colocs["balancerColocationCenter"].to_list()


[]

Should be empty, otherwise seen_colocs.csv needs updating with the missing colocs

In [14]:
data_grouped_by_colocation = (
    data
    .group_by("balancerColocationCenter")
    .agg(
        pl.col("balancerId").unique().count().alias("balancerCount")
    )
)
with_coords = data_grouped_by_colocation.join(colocs_with_coords, left_on="balancerColocationCenter", right_on="colocationCenter", how="left")
with_coords = with_coords.select(["balancerColocationCenter", "city", "lat", "lng", "balancerCount", "iso3"])
with_coords


balancerColocationCenter,city,lat,lng,balancerCount,iso3
str,str,f64,f64,u32,str
"""CPH""","""Copenhagen""",55.6805,12.5615,232,"""DNK"""
"""FOR""","""Fortaleza""",-3.7275,-38.5275,62,"""BRA"""
"""ISB""","""Islamabad""",33.6931,73.0639,15,"""PAK"""
"""ZAG""","""Zagreb""",45.8131,15.9772,11,"""HRV"""
"""DFW""","""Fort Worth""",32.7817,-97.3474,1432,"""USA"""
…,…,…,…,…,…
"""IAD""","""Washington""",38.9047,-77.0163,2277,"""USA"""
"""MEL""","""Melbourne""",-37.8142,144.9631,208,"""AUS"""
"""BKK""","""Bangkok""",13.7525,100.4942,258,"""THA"""


In [15]:
with_coords.write_csv("map_data.csv")